# Proyecto: Calculadora de presencialidad laboral

Actualmente, tengo un trabajo con un contrato de presencialidad 80/20 anual. El 80% corresponde al porcentaje de teletrabajo, y el 20% a la presencialidad en oficina. El 

Dicho esto, este proyecto nace como solución a los siguientes problemas:

- El porcentaje de teletrabajo es tan amplio que fácilmente puede llevar a un **incumplimiento del contrato**, por lo que exige tener cuidado de no sobrepasar ese 80% de trabajo telemático.

- **El sistema de fichaje de la empresa no proporciona información** relativa a los porcentajes de asistencia y teletrabajo.

- Para conocer el porcentaje real, es necesario **solicitar la información directamente al encargado de departamento**.

## Solución:
He creado el siguiente script en Python, permite calcular de forma automática el porcentaje de días teletrabajados en función de los registros contenidos en el CSV actualizable que hay en la misma carpeta:

In [83]:
import pandas as pd

#Damos la ruta de los datos:
csv_file = "data.csv"

In [3]:
#Cream os una función lambda para calcular porcentajes:
porcentaje = lambda x, y: (x*100) / y

#Generamos el DataFrame que vamos a manipular:
df_csv = pd.read_csv(csv_file, header=0)

In [4]:
#Recogemos los datos dependiendo de la presencialidad:
t_teletrabajo = (df_csv["Tipo"] == "Teletrabajo").sum()
t_presencial = (df_csv["Tipo"] == "Presencial").sum()

remaining = (df_csv["Tipo"]).isnull().sum()


total = t_teletrabajo + t_presencial

In [87]:
#Calculamos % de días de teletrabajo y presencial:
print(f"\nDías totales trabajados: {total}")

print(f"\n\tTeletrabajo {round(porcentaje(t_teletrabajo, total),2)}%, {t_teletrabajo} días.")
print(f"\n\tPresencial {round(porcentaje(t_presencial, total),2)}%, {t_presencial} días.")


Días totales trabajados: 224

	Teletrabajo 81.25%, 182 días.

	Presencial 18.75%, 42 días.


### Visualización de datos

Como ampliación del proyecto, se pretende pormenorizar en los datos obtenidos previamente. El objetivo es, poner en practica la extracción de datos mediante URL y la depuración de los mismos en formato HTML. Para ello, se pretende identificar qué días de la semana han sido los más recurrentes en la asistencia.

#### Solución:

In [85]:
import requests #Peticiones por HTTP
from bs4 import BeautifulSoup #Depuración de HTML
import matplotlib.pyplot as plt #Generación de gráficos

In [88]:
#Generamos un diccionario para hacer un recuento por días:
dic_diaspresenciales = {
    "Lunes":0,
    "Martes":0,
    "Miércoles":0,
    "Jueves":0,
    "Viernes":0
}

#Identificamos qué días del año ha habido presencialidad:
n_diaspresenciales = list(df_csv.loc[df_csv["Tipo"] == "Presencial", "Dia"].values) ########> COSA A DOCUMENTAR

#==================================
#RECOGEMOS LA INFORMACIÓN DE LA WEB
#==================================
url = "https://www.calendario-365.es/numeros-de-dias/2025.html#google_vignette"

##Texto de la URL
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)



#Recogemos solo el contenido de las tablas:
days = soup.find_all('tr')
#Eliminamos el encabezado para procesar:
for day in days[1:]:
    
    #Leemos el contenido día a día:
    text = list(day.find_all('td'))
    
    
    #Día del año:
    year_day = int(text[0].get_text())
    #Día de la semana:
    week_day= text[2].get_text().capitalize()

    #Ahora sumamos +1 al día de la semana que sabemos que es presencial:
    if year_day in n_diaspresenciales:
        dic_diaspresenciales[week_day] += 1 
    
print(f'Se han asistido {dic_diaspresenciales["Lunes"]} Lunes')
print(f'Se han asistido {dic_diaspresenciales["Martes"]} Martes')
print(f'Se han asistido {dic_diaspresenciales["Miércoles"]} Miércoles')
print(f'Se han asistido {dic_diaspresenciales["Jueves"]} Jueves')
print(f'Se han asistido {dic_diaspresenciales["Viernes"]} Viernes')

Se han asistido 2 Lunes
Se han asistido 9 Martes
Se han asistido 11 Miércoles
Se han asistido 16 Jueves
Se han asistido 4 Viernes
